In [5]:
# pip install chart_studio
# pip install datapane

Using legacy 'setup.py install' for retrying, since package 'wheel' is not installed.
    Running setup.py install for retrying: started
    Running setup.py install for retrying: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Mr-Phuc\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [8]:
%matplotlib inline
import pandas 
from sklearn.model_selection import train_test_split
import numpy as np
import time
import joblib
import Recommenders as Recommenders
import Evaluation as Evaluation

In [14]:
#Read useris, songid, listen
triplets_file = '10000.txt'
song_metadata_file = 'song_data.csv'
#Create dataframe for triplet file
song_df1 = pd.read_table(triplets_file, header = None)
song_df1.columns = ['user_id', 'song_id', 'listen_count']
#Dataframe for metadata file
song_df2 = pd.read_csv(song_metadata_file)

#Merge 2 data file
song_df = pd.merge(song_df1, song_df2.drop_duplicates(['song_id']),on="song_id",how='left')

In [16]:
song_df.head()
#Check length of data
print("Total number of elements:", len(song_df))
#Create subset of data
song_df = song_df.head(10000)

Total number of elements: 2000000


In [17]:
#Merge song title and song artist into one column
song_df['song'] = song_df['title'].map(str)+"-"+song_df['artist_name']

C:\Users\Mr-Phuc\AppData\Local\Temp/ipykernel_5224/1591507727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['song'] = song_df['title'].map(str)+"-"+song_df['artist_name']


In [19]:
#Number of unique users
users = song_df['user_id'].unique()
len(users)

365

In [20]:
#Number of unique songs:
songs = song_df['song'].unique()
len(songs)

5151

In [21]:
#Split data to train data and test data
train_data , test_data = train_test_split(song_df,test_size = 0.20, random_state = 0)
print(train_data.head(5))

                                       user_id             song_id  \
7389  94d5bdc37683950e90c56c9b32721edb5d347600  SOXNZOW12AB017F756   
9275  1012ecfd277b96487ed8357d02fa8326b13696a5  SOXHYVQ12AB0187949   
2995  15415fa2745b344bce958967c346f2a89f792f63  SOOSZAZ12A6D4FADF8   
5316  ffadf9297a99945c0513cd87939d91d8b602936b  SOWDJEJ12A8C1339FE   
356   5a905f000fc1ff3df7ca807d57edb608863db05d  SOAMPRJ12A8AE45F38   

      listen_count                 title  \
7389             2      Half Of My Heart   
9275             1  The Beautiful People   
2995             1     Sanctify Yourself   
5316             4     Heart Cooks Brain   
356             20                 Rorol   

                                                release      artist_name  \
7389                                     Battle Studies       John Mayer   
9275             Antichrist Superstar (Ecopac Explicit)   Marilyn Manson   
2995                             Glittering Prize 81/92     Simple Minds   
5316  Ever

In [25]:
# Simple popularity based recommender
pm = Recommenders.popularity_recommender()
pm.create(train_data,'user_id','song')

In [27]:
# Using recommender to make some recommendation
user_id = users[35]
pm.recommend(user_id)

,user_id,song,score,Rank
3194,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Sehr kosmisch-Harmonia,37,1.0
4083,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Undo-Björk,27,2.0
931,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Dog Days Are Over (Radio Edit)-Florence + The ...,24,3.0
4443,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,You're The One-Dwight Yoakam,24,4.0
3034,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Revelry-Kings Of Leon,21,5.0
3189,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Secrets-OneRepublic,21,6.0
4112,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Use Somebody-Kings Of Leon,21,7.0
1207,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Fireflies-Charttraxx Karaoke,20,8.0
1577,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Hey_ Soul Sister-Train,19,9.0
1626,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [28]:
is_model = Recommenders.item_similarity_recommender()
is_model.create(train_data,'user_id','song')

In [29]:
user_item = is_model.get_user_items(user_id)
print("Users history music data:\n",user_item)
print("Recommendations for User id",user_id,":")
is_model.recommend(user_id)

Users history music data:
 ['Unite (2009 Digital Remaster)-Beastie Boys', 'Long Black Veil-DAVE MATTHEWS BAND', 'Kryptonite-3 Doors Down', 'Fortunate Son-Creedence Clearwater Revival', 'Long As I Can See The Light-Creedence Clearwater Revival', 'Human After All-Daft Punk', 'Have You Ever Seen The Rain-Creedence Clearwater Revival', 'The Prime Time Of Your Life-Daft Punk', 'Gotas De Agua Dulce-Juanes', "Paradise City-Guns N' Roses", 'New Divide (Album Version)-Linkin Park', 'Born On The Bayou-Creedence Clearwater Revival', 'Down On The Corner-Creedence Clearwater Revival', 'Gloomy-Creedence Clearwater Revival']
Recommendations for User id 90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f :
No. of unique songs for the user: 14
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :2088


,user_id,song,score,rank
0,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Television Rules The Nation-Daft Punk,0.071429,1
1,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Voyager-Daft Punk,0.071429,2
2,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Aerodynamic-Daft Punk,0.071429,3
3,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Alive-Daft Punk,0.071429,4
4,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Emotion-Daft Punk,0.071429,5
5,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Short Circuit-Daft Punk,0.071429,6
6,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Meanwhile_ Rick James...-Cake,0.059524,7
7,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Too Long-Daft Punk,0.059524,8
8,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Superhereos-Daft Punk,0.059524,9
9,90dbbbb47979f46ccdbc413d6b6e31cdb55aa58f,Robot Rock-Daft Punk,0.059524,10
